In [1]:
from ipywidgets import interactive
from ipywidgets import FloatSlider

In [2]:
import module.plot_Jouk_helper as hpl
hpl.define_mplt_par()

In [3]:
Vinf = 1.5

In [4]:
def f1(xpos,ypos,AoA,Gamma):
    from numpy import deg2rad
    alpha = deg2rad(AoA)
    hpl.plot_phi_psi_cylinder(Vinf=Vinf,alpha=alpha,center=[-xpos,ypos],Gamma=Gamma,nr=100,nt=720,apply_Kutta=False)

xw = FloatSlider(min=0.005,max=0.3,step=0.005,value=0.1,continuous_update=False)
yw = FloatSlider(min=0.0,max=0.5,step=0.01,value=0.1,continuous_update=False)
Aw = FloatSlider(min=0.0,max=12.0,step=0.5,value=0.0,continuous_update=False)
Gw = FloatSlider(min=0.0,max=8.0,step=0.1,value=0.0,continuous_update=False)

interactive_plot = interactive(f1,xpos=xw,ypos=yw,AoA=Aw,Gamma=Gw,continuous_update=False)
# output = interactive_plot.children[-1]
# output.layout.height = '350px'
interactive_plot



interactive(children=(FloatSlider(value=0.1, continuous_update=False, description='xpos', max=0.3, min=0.005, …

In [5]:
def f2(xpos,ypos,AoA):
    from numpy import deg2rad
    alpha = deg2rad(AoA)
    hpl.plot_vel_cylinder(Vinf=Vinf,alpha=alpha,center=[-xpos,ypos],nr=100,nt=720)
    hpl.plot_press_cylinder(Vinf=Vinf,alpha=alpha,center=[-xpos,ypos],nr=100,nt=720)

xw = FloatSlider(min=0.005,max=0.3,step=0.005,value=0.1,continuous_update=False)
yw = FloatSlider(min=0.0,max=0.5,step=0.01,value=0.1,continuous_update=False)
Aw = FloatSlider(min=0.0,max=12.0,step=0.5,value=0.0,continuous_update=False)

interactive_plot = interactive(f2,xpos=xw,ypos=yw,AoA=Aw,continuous_update=False)
# output = interactive_plot.children[-1]
# output.layout.height = '350px'
interactive_plot



interactive(children=(FloatSlider(value=0.1, continuous_update=False, description='xpos', max=0.3, min=0.005, …

In [6]:
def f3(xpos,ypos,AoA):
    from numpy import deg2rad
    alpha = deg2rad(AoA)
    hpl.plot_Cp_peau(Vinf=Vinf,alpha=alpha,center=[-xpos,ypos],nr=100,nt=720)
    
xw = FloatSlider(min=0.005,max=0.3,step=0.005,value=0.1,continuous_update=False)
yw = FloatSlider(min=0.0,max=0.5,step=0.01,value=0.1,continuous_update=False)
Aw = FloatSlider(min=0.0,max=12.0,step=0.5,value=0.0,continuous_update=False)
interactive_plot = interactive(f3,xpos=xw,ypos=yw,AoA=Aw)
# output = interactive_plot.children[-1]
# output.layout.height = '350px'
interactive_plot



interactive(children=(FloatSlider(value=0.1, continuous_update=False, description='xpos', max=0.3, min=0.005, …